In [ ]:
# If you do not have these datafiles yet, use the adaptive_zoning_preprocess_EW_data
# notebook to download and preprocess the data.

import geopandas as gpd
import pandas as pd

centroid_data = gpd.read_parquet('data/centroid_data.parquet')
commuting_data = pd.read_parquet('data/commuting_data.parquet')

In [ ]:
from shapely.geometry import Point

# Create dictionaries for quick lookup of MSOA coordinates
# can probably be done more efficently with pd merge
msoa_locations = centroid_data.set_index('msoa11cd')['geometry'].to_dict()

weighted_distances = sum(
    msoa_locations[origin].distance(msoa_locations[destination]) * commuters
    for origin, destination, commuters in zip(
        commuting_data['Area of residence'],
        commuting_data['Area of workplace'],
        commuting_data['Commuters']
    )
)

total_commuters = sum((commuters for commuters in commuting_data['Commuters']))
   
average_distance = weighted_distances / total_commuters
print(average_distance)

In [ ]:
import numpy as np
from doubly_constrained_spatial_interaction_model import distance_matrix_from_points
from doubly_constrained_spatial_interaction_model import doubly_constrained
from doubly_constrained_spatial_interaction_model import calibrate_doubly_constrained

# divide units by a 1000 so distances are in km and beta is in a more typical range
points = np.array([(point.x/1000, point.y/1000) for point in centroid_data.geometry])
pop =  centroid_data['Residential population'].to_numpy()
emp =  centroid_data['Workplace population'].to_numpy() 
distance_matrix = distance_matrix_from_points(points)
average_distance = 14464.171110830612 / 1000 # taken from the cell above
beta = calibrate_doubly_constrained(pop,emp, distance_matrix, average_distance)
trips, av_distance, a, b = doubly_constrained(pop, emp, distance_matrix, beta,verbose = True)
print("average distance: ", av_distance, " beta: ", beta)